# Enhancing Invoice Processing with Expert Tools

## Step 1: Creating the Invoice Categorization Expert
We’ll start by defining an expert who specializes in classifying expenditures. This expert will take a one-sentence description of the invoice and return the best-fitting category from our predefined list.




In [ ]:
@register_tool(tags=["invoice_processing", "categorization"])
def categorize_expenditure(action_context: ActionContext, description: str) -> str:
    """
    Categorize an invoice expenditure based on a short description.

    Args:
        description: A one-sentence summary of the expenditure.

    Returns:
        A category name from the predefined set of 20 categories.
    """
    categories = [
        "Office Supplies", "IT Equipment", "Software Licenses", "Consulting Services",
        "Travel Expenses", "Marketing", "Training & Development", "Facilities Maintenance",
        "Utilities", "Legal Services", "Insurance", "Medical Services", "Payroll",
        "Research & Development", "Manufacturing Supplies", "Construction", "Logistics",
        "Customer Support", "Security Services", "Miscellaneous"
    ]

    return prompt_expert(
        action_context=action_context,
        description_of_expert="A senior financial analyst with deep expertise in corporate spending categorization.",
        prompt=f"Given the following description: '{description}', classify the expense into one of these categories:\n{categories}"
    )


##Step 2: Creating the Purchasing Rules Expert

To ensure that our **purchasing rules expert** is always using the latest policy guidelines, we will implement it to **read the purchasing rules from a file on disk** before validating an invoice. This approach makes our system **dynamic**—we don’t need to hardcode the rules into the agent, and they can be updated easily without modifying the code. Also, we don’t need a special format for the rules, the human-readable policy becomes the logic!

First, let’s assume that the purchasing rules are stored in a simple **text file** located at `"config/purchasing_rules.txt"`. The expert will **read these rules, insert them into the prompt**, and then validate the invoice accordingly.

`"config/purchasing_rules.txt"` could look like below:

1. All purchases over 5000 USD require pre-approval.
2. IT equipment purchases must be from approved vendors.
3. Travel expenses must include a justification.
4. Consulting fees over 10,000 USD require an SOW (Statement of Work).

In [ ]:
@register_tool(tags=["invoice_processing", "validation"])
def check_purchasing_rules(action_context: ActionContext, invoice_data: dict) -> dict:
    """
    Validate an invoice against company purchasing policies.

    Args:
        invoice_data: Extracted invoice details, including vendor, amount, and line items.

    Returns:
        A dictionary indicating whether the invoice is compliant, with explanations.
    """
    # Load the latest purchasing rules from disk
    rules_path = "config/purchasing_rules.txt"

    try:
        with open(rules_path, "r") as f:
            purchasing_rules = f.read()
    except FileNotFoundError:
        purchasing_rules = "No rules available. Assume all invoices are compliant."

    return prompt_expert(
        action_context=action_context,
        description_of_expert="A corporate procurement compliance officer with extensive knowledge of purchasing policies.",
        prompt=f"""
        Given this invoice data: {invoice_data}, check whether it complies with company purchasing rules.
        The latest purchasing rules are as follows:

        {purchasing_rules}

        Identify any violations or missing requirements. Respond with:
        - "compliant": true or false
        - "issues": A brief explanation of any problems found
        """
    )

##Generating Structured Responses Rather than Free Text

While this free-text reasoning is powerful, it isn’t always enough. There are situations where the results of an evaluation need to be captured in a format that other systems can work with. If an organization wants to log purchasing violations in a database, trigger automated workflows based on compliance status, or generate structured reports on spending patterns, then a loosely written explanation from an LLM—however insightful—won’t be enough. The output needs to follow a predictable structure so that it can be systematically processed.

To make this possible, we can shift from a purely natural language response to a structured JSON format. Instead of returning an open-ended explanation, we direct the LLM to produce a response that includes two key elements: a boolean flag that indicates whether the invoice is compliant (`true` or `false`), and a structured field that provides a justification for the decision. By using `prompt_llm_for_json`, we ensure that the response conforms to this expected format, eliminating ambiguity and making it easy to integrate with downstream systems.

The modification to our validation tool is straightforward. Instead of returning a free-text analysis, we define a schema that enforces the structured response. The expert tool reads the purchasing rules from disk, provides those rules as context to the LLM, and requests a JSON-formatted response that aligns with the schema.

In [ ]:
@register_tool(tags=["invoice_processing", "validation"])
def check_purchasing_rules(action_context: ActionContext, invoice_data: dict) -> dict:
    """
    Validate an invoice against company purchasing policies, returning a structured response.

    Args:
        invoice_data: Extracted invoice details, including vendor, amount, and line items.

    Returns:
        A structured JSON response indicating whether the invoice is compliant and why.
    """
    rules_path = "config/purchasing_rules.txt"

    try:
        with open(rules_path, "r") as f:
            purchasing_rules = f.read()
    except FileNotFoundError:
        purchasing_rules = "No rules available. Assume all invoices are compliant."

    validation_schema = {
        "type": "object",
        "properties": {
            "compliant": {"type": "boolean"},
            "issues": {"type": "string"}
        }
    }

    return prompt_llm_for_json(
        action_context=action_context,
        schema=validation_schema,
        prompt=f"""
        Given this invoice data: {invoice_data}, check whether it complies with company purchasing rules.
        The latest purchasing rules are as follows:

        {purchasing_rules}

        Respond with a JSON object containing:
        - `compliant`: true if the invoice follows all policies, false otherwise.
        - `issues`: A brief explanation of any violations or missing requirements.
        """
    )

##Step 3: Updating the Invoice Processing Agent
Now that we have our expert tools, we update the Invoice Processing Agent to intelligently decide:

- When to extract invoice data
- When to categorize expenditures
- When to validate invoices
- How to store the processed information

##Full Agent Code:

In [ ]:
def create_invoice_agent():
    # Create action registry with invoice tools
    action_registry = PythonActionRegistry()

    # Define invoice processing goals
    goals = [
        Goal(
            name="Persona",
            description="You are an Invoice Processing Agent, specialized in handling invoices efficiently."
        ),
        Goal(
            name="Process Invoices",
            description="""
            Your goal is to process invoices accurately. For each invoice:
            1. Extract key details such as vendor, amount, and line items.
            2. Generate a one-sentence summary of the expenditure.
            3. Categorize the expenditure using an expert.
            4. Validate the invoice against purchasing policies.
            5. Store the processed invoice with categorization and validation status.
            6. Return a summary of the invoice processing results.
            """
        )
    ]

    # Define agent environment
    environment = PythonEnvironment()

    return Agent(
        goals=goals,
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=action_registry,
        generate_response=generate_response,
        environment=environment
    )